In [23]:

import os

# os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''

In [35]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain import PromptTemplate, HuggingFaceHub
model = "meta-llama/Llama-3.2-1B-Instruct"
model = HuggingFaceEndpoint(
        repo_id=model,
        max_length=256,
        temperature=0.5,
    )

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


In [36]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

In [37]:
question = "Given a Student table with columns: Name, Roll No. and Grade. write SQL Query to Display name and roll no. of students who have grade A."

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

In [38]:
llm_chain = prompt | model 
print(llm_chain.invoke({"question": question}))

 We need to select the name and roll no. of students who have grade A. We can use the following SQL query to display the desired information.

```sql
SELECT Name, Roll No. 
FROM Student 
WHERE Grade = 'A';
```

Explanation:

- We start with the `SELECT` statement, which specifies what columns we want to retrieve from the database.
- We use the `Name` and `Roll No.` columns to display the student's name and roll no.
- We use the `WHERE` clause to filter the data. In this case, we want to retrieve students with a grade of 'A'. The `WHERE` clause is used to specify the condition for which rows to include in the result set.
- Finally, we use the `FROM` statement to specify the table from which to retrieve the data, which is the `Student` table in this case.

When you run this query on the `Student` table, you should see the names and roll nos. of students who have a grade of 'A'. For example, if the table contains the following data:

| Name | Roll No. | Grade |
|------|---------|-------|


In [40]:
from langchain_cohere import ChatCohere, create_sql_agent 
from langchain_community.utilities import SQLDatabase

In [47]:
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
import sqlite3

dbfilepath = "/home/himanshulalarya/project/himanshu/code/ChatSQL/student.db"
creator = lambda: sqlite3.connect(f"file:{dbfilepath}?mode=ro", uri=True)

In [48]:
sqldb =  SQLDatabase(create_engine("sqlite:///", creator=creator))

In [49]:
sqldb

In [61]:
sqlite_uri = 'sqlite:///' +dbfilepath
db = SQLDatabase.from_uri(sqlite_uri)

In [94]:
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)


In [95]:
def get_schema(_):
    print(type(sqldb))
    schema = sqldb.get_table_info()
    return schema


In [96]:
get_schema(db)

<class 'langchain_community.utilities.sql_database.SQLDatabase'>


'\nCREATE TABLE "STUDENT" (\n\t"NAME" VARCHAR(25), \n\t"CLASS" VARCHAR(25), \n\t"SECTION" VARCHAR(25), \n\t"MARKS" INTEGER\n)\n\n/*\n3 rows from STUDENT table:\nNAME\tCLASS\tSECTION\tMARKS\nKrish\tData Science\tA\t90\nJohn\tData Science\tB\t100\nMukesh\tData Science\tA\t86\n*/'

In [97]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm = model

sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)


In [107]:
user_question = 'show me the students which has got marks greater than or equal to 90'
response = sql_chain.invoke({"question": user_question})
response
# 'SELECT COUNT(*) AS TotalAlbums\nFROM Album;'


<class 'langchain_community.utilities.sql_database.SQLDatabase'>


' \nSELECT * FROM STUDENT WHERE MARKS >= 90\n\nThis query will return all the rows from the STUDENT table where the MARKS column is greater than or equal to 90. \n\nNote: The query does not include the name, class, and section columns, as they are not relevant to the condition of having MARKS greater than or equal to 90. \n\nHowever, if you want to include these columns in the result set, you can use the following query:\n\nSELECT * FROM STUDENT WHERE MARKS >= 90\n\nThis query will return all the rows from the STUDENT table where the MARKS column is greater than or equal to 90. The result set will include the NAME, CLASS, and SECTION columns. \n\nFor example, the result set might look like this:\n\nNAME\tCLASS\tSECTION\tMARKS\nKrish\tData Science\tA\t90\nJohn\tData Science\tB\t100\nMukesh\tData Science\tA\t86\n\nIn this case, all three students have MARKS greater than or equal to 90. \n\nIf you want to include only the students with MARKS greater than 90, you can modify the query as fo

In [99]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)


In [100]:
def run_query(query):
    # Simple check for multiple statements (this is a basic approach and may need refinement)
    if query.count(';') > 1:
        raise ValueError("Multiple SQL statements detected. Please execute one statement at a time.")
    return db.run(query)


In [104]:
full_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
        schema=get_schema,
        response=lambda vars: run_query(vars["query"]),
    )
    | prompt_response
    | model
)


In [105]:
user_question = 'show me all the content of student table?'
full_chain.invoke({"question": user_question})

# 'There are 347 albums in the database.'


Exception closing connection <sqlite3.Connection object at 0x7f85d031a040>
Traceback (most recent call last):
  File "/home/himanshulalarya/miniconda3/envs/test/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 376, in _close_connection
    self._dialect.do_close(connection)
  File "/home/himanshulalarya/miniconda3/envs/test/lib/python3.10/site-packages/sqlalchemy/engine/default.py", line 709, in do_close
    dbapi_connection.close()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 140212732687936 and this is thread id 140212753663552.


<class 'langchain_community.utilities.sql_database.SQLDatabase'>


Exception closing connection <sqlite3.Connection object at 0x7f85d3ab1d40>
Traceback (most recent call last):
  File "/home/himanshulalarya/miniconda3/envs/test/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 376, in _close_connection
    self._dialect.do_close(connection)
  File "/home/himanshulalarya/miniconda3/envs/test/lib/python3.10/site-packages/sqlalchemy/engine/default.py", line 709, in do_close
    dbapi_connection.close()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 140226521220928 and this is thread id 140212732687936.


<class 'langchain_community.utilities.sql_database.SQLDatabase'>


Warning: You can only execute one statement at a time.

In [50]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=sqldb, llm=model)

In [ ]:
toolkit.get_tools()

In [60]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool


def execute_query(state: State):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    return {"result": execute_query_tool.invoke(state["query"])}

NameError: name 'State' is not defined

In [56]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
prompt_template.messages[0].pretty_print()

/home/himanshulalarya/miniconda3/envs/test/lib/python3.10/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


================================ System Message ================================

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to th

In [59]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

system_message = prompt_template.format(dialect="SQLite", top_k=3)
agent_executor = create_react_agent(model, toolkit, state_modifier=system_message)

ValueError: The first argument must be a string or a callable with a __name__ for tool decorator. Got <class 'tuple'>

In [90]:
def process_sql_response(response):
    # Extract SQL query
    sql_match = re.search(r'SELECT.*?;', response, re.IGNORECASE | re.DOTALL)
    if sql_match:
        sql_query = sql_match.group(0).strip()
        st.code(sql_query, language='sql')
    else:
        st.write("No SQL query found in the response.")
    
    # Extract table data
    table_match = re.search(r'(NAME.*?)(?=\n\n|\Z)', response, re.DOTALL)
    if table_match:
        table_data = table_match.group(1).strip().split('\n')
        headers = table_data[0].split('\t')
        rows = [row.split('\t') for row in table_data[1:]]
        
        df = pd.DataFrame(rows, columns=headers)
        st.dataframe(df)
    else:
        st.write("No table output found in the response.")
    
    # Display any additional information
    additional_info = re.sub(r'SELECT.*?;|NAME.*', '', response, flags=re.DOTALL).strip()
    if additional_info:
        st.write("Additional Information:")
        st.write(additional_info)

In [91]:
import re
import streamlit as st
import pandas as pd

process_sql_response(response)

2025-01-20 00:05:28.487 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-20 00:05:28.533 
  command:

    streamlit run /home/himanshulalarya/miniconda3/envs/test/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-01-20 00:05:28.534 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-20 00:05:28.539 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-20 00:05:28.539 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-20 00:05:28.540 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-20 00:05:28.540 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-20 00:05:2